In [8]:
%pip install -U langchain-qdrant langchain-experimental

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_upstage import UpstageDocumentParseLoader
from dotenv import load_dotenv, find_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv(find_dotenv())

COLLECTION_NAME = "team_project"


embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large-passage"
)


loader = UpstageDocumentParseLoader(
        "./TeamProjectPlan.pdf", 
        output_format="markdown"
)
docs = loader.load()


In [4]:
from langchain_experimental.text_splitter import SemanticChunker
text_splitter = SemanticChunker(
    embeddings, 
    breakpoint_threshold_type="percentile"
)



splits = text_splitter.split_documents(docs)

In [5]:
for doc in splits:
    # 1) 모든 청크에 공통적으로 들어갈 정보
    doc.metadata["workspace_id"] = 1

In [6]:
from langchain_qdrant import QdrantVectorStore
import os
# 4. Qdrant에 저장 (Ingestion)

vectorstore = QdrantVectorStore.from_documents(
    documents=splits,
    embedding=embeddings,
    url=os.getenv("QDRANT_URL"),
    collection_name=COLLECTION_NAME,
    force_recreate=True  # 주의: True면 기존 데이터를 지우고 새로 만듭니다. (업데이트 시 유용)
)

In [7]:
query_embedding = UpstageEmbeddings(model="embedding-query")
# 로컬에서 
loaded_vectorstore = QdrantVectorStore.from_existing_collection(
    embedding=query_embedding,
    collection_name=COLLECTION_NAME,
    url=os.getenv("QDRANT_URL")
)

In [13]:
from qdrant_client import models
qdrant_filter = models.Filter(
    must=[
        models.FieldCondition(
            key="workspace_id",          # 메타데이터 키 이름
            match=models.MatchValue(value=1) # 찾으려는 값
        )
    ]
)

In [23]:
results_with_scores = loaded_vectorstore.similarity_search_with_relevance_scores(
    query="해당 프로젝트의 기획 의도가 무엇인가요?",
    k=3,                   # retriever의 "k"
    filter=qdrant_filter   # retriever의 "filter"
)

# 3. 결과 출력 (문서와 점수가 튜플로 나옵니다)
print(f"🔍 검색 결과: {len(results_with_scores)}건\n")

for doc, score in results_with_scores:
    print(f"[Score]: {score:.4f}") # 소수점 4자리까지 출력
    print(f"[Content]: {doc.page_content[:100]}...") # 내용 일부
    print(f"[Metadata]: {doc.metadata}")
    print("-" * 50)

🔍 검색 결과: 0건



In [ ]:
# 2. 검색 실행 (동기)
query = "해당 프로젝트의 기획의도"
docs = results_with_scores.invoke(query)

# 3. 결과 확인
for doc in docs:
    print(f"=========================")
    print(f"[내용]: {doc.page_content}")
    print(f"[메타데이터]: {doc.metadata}")

No relevant docs were retrieved using the relevance score threshold 0.3
